In [1]:
from typing import Union, List

class RingElement:
    """Базовый класс для элементов колец: целых чисел и полиномов."""

    def __init__(self, data: Union[int, List[float]]):
        """
        data:
        - если это целое число → элемент кольца Z;
        - если это список коэффициентов [a0, a1, ..., an] → полином a0 + a1*x + ... + an*x^n.
        """
        self.data = data
        self.is_polynomial = isinstance(data, list)

    def __repr__(self) -> str:
        if self.is_polynomial:
            terms = []
            for i, c in enumerate(self.data):
                if c != 0:
                    if i == 0:
                        terms.append(str(c))
                    elif i == 1:
                        terms.append(f"{c}*x")
                    else:
                        terms.append(f"{c}*x^{i}")
            return " + ".join(terms) if terms else "0"
        return str(self.data)

    def __eq__(self, other):
        return self.data == other.data and self.is_polynomial == other.is_polynomial

def gcd_ring_elements(elements: List[RingElement]) -> RingElement:
    """
    Возвращает порождающий главного идеала, порождённого заданными элементами.
    - Для Z: НОД целых чисел.
    - Для K[x]: НОД полиномов (с коэффициентами в поле K).
    """
    if not elements:
        raise ValueError("Список элементов не может быть пустым")

    # Проверяем, что все элементы одного типа
    first_type = elements[0].is_polynomial
    for elem in elements:
        if elem.is_polynomial != first_type:
            raise ValueError("Все элементы должны быть одного типа (целые числа или полиномы)")

    if first_type:
        # Случай полиномов K[x]
        return _gcd_polynomials(elements)
    else:
        # Случай целых чисел Z
        return _gcd_integers(elements)

def _gcd_integers(elements: List[RingElement]) -> RingElement:
    """НОД для целых чисел с использованием алгоритма Евклида"""
    numbers = [abs(elem.data) for elem in elements]  # НОД всегда неотрицательный

    if len(numbers) == 1:
        return RingElement(numbers[0])

    # Вычисляем НОД последовательно
    current_gcd = numbers[0]
    for num in numbers[1:]:
        current_gcd = _gcd_two_integers(current_gcd, num)
        if current_gcd == 1:
            break

    return RingElement(current_gcd)

def _gcd_two_integers(a: int, b: int) -> int:
    """НОД двух целых чисел по алгоритму Евклида"""
    a, b = abs(a), abs(b)
    while b != 0:
        a, b = b, a % b
    return a

def _gcd_polynomials(elements: List[RingElement]) -> RingElement:
    """НОД для полиномов с использованием алгоритма Евклида"""
    polynomials = [elem.data for elem in elements]

    # Убираем ведущие нули
    polynomials = [_remove_leading_zeros(poly) for poly in polynomials]

    if len(polynomials) == 1:
        return RingElement(_make_monic(polynomials[0]))

    # Вычисляем НОД последовательно
    current_gcd = polynomials[0]
    for poly in polynomials[1:]:
        current_gcd = _gcd_two_polynomials(current_gcd, poly)
        if len(current_gcd) == 1 and abs(current_gcd[0]) < 1e-10:  # НОД = 0
            current_gcd = [0]
            break

    return RingElement(_make_monic(current_gcd))

def _gcd_two_polynomials(a: List[float], b: List[float]) -> List[float]:
    """НОД двух полиномов по алгоритму Евклида"""
    a = _remove_leading_zeros(a)
    b = _remove_leading_zeros(b)

    # Если один из полиномов нулевой, возвращаем другой
    if len(a) == 1 and abs(a[0]) < 1e-10:
        return b
    if len(b) == 1 and abs(b[0]) < 1e-10:
        return a

    poly1, poly2 = a, b

    while True:
        # Делим poly1 на poly2
        quotient, remainder = _polynomial_division(poly1, poly2)

        # Если остаток нулевой, то poly2 - НОД
        if len(remainder) == 1 and abs(remainder[0]) < 1e-10:
            return poly2

        poly1, poly2 = poly2, remainder

def _polynomial_division(dividend: List[float], divisor: List[float]) -> tuple:
    """Деление полиномов с остатком"""
    dividend = _remove_leading_zeros(dividend)
    divisor = _remove_leading_zeros(divisor)

    if len(divisor) == 1 and abs(divisor[0]) < 1e-10:
        raise ValueError("Деление на нулевой полином")

    # Степень делимого меньше степени делителя
    if len(dividend) < len(divisor):
        return [0], dividend

    quotient_degree = len(dividend) - len(divisor)
    quotient = [0.0] * (quotient_degree + 1)
    remainder = dividend.copy()

    divisor_lead = divisor[-1]  # Старший коэффициент делителя

    for i in range(quotient_degree, -1, -1):
        if len(remainder) >= len(divisor) + i:
            factor = remainder[len(remainder) - 1] / divisor_lead
            quotient[i] = factor

            # Вычитаем factor * divisor * x^i из remainder
            for j in range(len(divisor)):
                idx = i + j
                if idx < len(remainder):
                    remainder[idx] -= factor * divisor[j]

    # Округляем очень маленькие числа до нуля
    remainder = _remove_leading_zeros([round(x, 10) for x in remainder])
    quotient = _remove_leading_zeros([round(x, 10) for x in quotient])

    return quotient, remainder

def _remove_leading_zeros(poly: List[float]) -> List[float]:
    """Убирает ведущие нули из полинома"""
    if not poly:
        return [0]

    # Находим последний ненулевой коэффициент
    last_nonzero = len(poly) - 1
    while last_nonzero >= 0 and abs(poly[last_nonzero]) < 1e-10:
        last_nonzero -= 1

    if last_nonzero < 0:
        return [0]  # Все коэффициенты нулевые

    return poly[:last_nonzero + 1]

def _make_monic(poly: List[float]) -> List[float]:
    """Делает полином мономом (старший коэффициент = 1)"""
    poly = _remove_leading_zeros(poly)

    if len(poly) == 1 and abs(poly[0]) < 1e-10:
        return [0]  # Нулевой полином

    lead_coef = poly[-1]
    if abs(lead_coef - 1.0) < 1e-10:
        return poly  # Уже моном

    # Делим все коэффициенты на старший
    return [coef / lead_coef for coef in poly]

# Тесты
def test_gcd():
    print("ТЕСТЫ ДЛЯ КОЛЕЦ ГЛАВНЫХ ИДЕАЛОВ")

    # Целые числа
    print("1. Тест для целых чисел Z:")
    nums = [RingElement(12), RingElement(18), RingElement(24)]
    gcd_result = gcd_ring_elements(nums)
    print(f"   НОД(12, 18, 24) = {gcd_result}")
    print(f"   Ожидаемый результат: 6")

    # Простые числп
    nums2 = [RingElement(17), RingElement(19)]
    gcd_result2 = gcd_ring_elements(nums2)
    print(f"   НОД(17, 19) = {gcd_result2}")
    print(f"   Ожидаемый результат: 1")

    # Полиномы
    print("\n2. Тест для полиномов K[x]:")
    # (x^2 - 1) и (x - 1)
    poly1 = RingElement([-1, 0, 1])  # x^2 - 1
    poly2 = RingElement([-1, 1])     # x - 1
    gcd_poly = gcd_ring_elements([poly1, poly2])
    print(f"   НОД(x^2 - 1, x - 1) = {gcd_poly}")
    print(f"   Ожидаемый результат: x - 1")

    # Полиномы без общих делителей
    poly3 = RingElement([1, 1])      # x + 1
    poly4 = RingElement([1, -1])     # x - 1
    gcd_poly2 = gcd_ring_elements([poly3, poly4])
    print(f"   НОД(x + 1, x - 1) = {gcd_poly2}")
    print(f"   Ожидаемый результат: 1")

    # Три полинома
    poly5 = RingElement([0, 2, -2])  # 2x^2 - 2x
    poly6 = RingElement([-1, 1])     # x - 1
    poly7 = RingElement([0, 0, 1])   # x^2
    gcd_poly3 = gcd_ring_elements([poly5, poly6, poly7])
    print(f"   НОД(2x^2 - 2x, x - 1, x^2) = {gcd_poly3}")

    print("\n3. Проверка свойств главных идеалов:")
    print("В Z и K[x] все идеалы являются главными")
    print("НОД порождает идеал, образованный данными элементами")
    print("Алгоритм Евклида гарантирует нахождение НОД")

if __name__ == "__main__":
    test_gcd()

    print("1. Кольца Z и K[x] являются кольцами главных идеалов")
    print("2. Любой идеал в них порождается одним элементом - НОД")
    print("3. Алгоритм Евклида эффективно находит порождающий элемент")
    print("4. Для полиномов результат приводится к моному")

ТЕСТЫ ДЛЯ КОЛЕЦ ГЛАВНЫХ ИДЕАЛОВ
1. Тест для целых чисел Z:
   НОД(12, 18, 24) = 6
   Ожидаемый результат: 6
   НОД(17, 19) = 1
   Ожидаемый результат: 1

2. Тест для полиномов K[x]:
   НОД(x^2 - 1, x - 1) = -1.0 + 1.0*x
   Ожидаемый результат: x - 1
   НОД(x + 1, x - 1) = 1.0
   Ожидаемый результат: 1
   НОД(2x^2 - 2x, x - 1, x^2) = 1.0

3. Проверка свойств главных идеалов:
   - В Z и K[x] ВСЕ идеалы являются главными
   - НОД порождает идеал, образованный данными элементами
   - Алгоритм Евклида гарантирует нахождение НОД
1. Кольца Z и K[x] являются кольцами главных идеалов
2. Любой идеал в них порождается одним элементом - НОД
3. Алгоритм Евклида эффективно находит порождающий элемент
4. Для полиномов результат приводится к монic виду
